# Project 3

## Project Overview

- **Dataset(s) to be used:**  
  - **2023 Yellow Taxi Trip Data:** https://data.cityofnewyork.us/resource/4b4i-vvec.json  
  - **NYC Taxi Zones:** https://data.cityofnewyork.us/resource/8meu-9t5y.json  

- **Analysis question:**  
  For yellow taxi trips in January 2023 that begin in Manhattan, are paid by credit card, and fall within typical trip distances, how do tip percentages vary by hour of day and day of week?  
  To make a fair comparison, the analysis focuses only on hours where both weekday and weekend trips are observed**.

- **Columns that will be used:**
  - From trip data:  
    `tpep_pickup_datetime`, `tpep_dropoff_datetime`,  
    `passenger_count`, `trip_distance`,  
    `pulocationid`, `fare_amount`, `tip_amount`,  
    `total_amount`, `payment_type`
  - From Taxi Zones (after merging on `locationid`):  
    `borough` (used to identify Manhattan pickups)

- **Columns used for merging:**  
  - Trip data: `pulocationid` / `PULocationID`  
  - Taxi Zones: `locationid`

- **Hypothesis:**  
  After restricting the analysis to shared hours, weekend trips, especially during evening hours, will show higher average tip percentages than weekday trips, even when focusing on typical Manhattan taxi rides paid by credit card.


### Step 1

**Load January 2023 yellow taxi data**

In this step I use the NYC Open Data (Socrata) API to load a sample of yellow taxi trips where the pickup datetime is in **January 2023**. This will be the base dataset for the rest of the analysis.

- Create a Socrata client for `data.cityofnewyork.us` using the `sodapy` library.
- Query the **2023 Yellow Taxi Trip Data** (`4b4i-vvec`) with a `$where` filter to keep trips with `tpep_pickup_datetime` between `2023-01-01` and `2023-02-01`.
- Select the main columns needed for tipping and timing (pickup/dropoff datetimes, trip distance, location IDs, fare, tip, total, and payment type).
- Limit the result to a manageable sample size (for example, 100,000 trips).
- Convert datetime and numeric columns to appropriate types.
- Use `.head()`, `.info()`, and `.describe()` to confirm the number of rows, column types, and rough ranges for fares, tips, and distances.


In [166]:
# ensure the visualizations render properly across VSCode, Jupyter Book, etc.
# https://plotly.com/python/renderers/

import plotly.io as pio
pio.renderers.default = "notebook_connected+plotly_mimetype"

import pandas as pd
from sodapy import Socrata

# Unauthenticated client for public NYC Open Data
client = Socrata("data.cityofnewyork.us", None)

TRIP_DATASET_ID = "4b4i-vvec"  # 2023 Yellow Taxi Trip Data

# Columns to pull
trip_select = (
    "tpep_pickup_datetime,"
    "tpep_dropoff_datetime,"
    "passenger_count,"
    "trip_distance,"
    "pulocationid,"
    "dolocationid,"
    "fare_amount,"
    "extra,"
    "mta_tax,"
    "tip_amount,"
    "tolls_amount,"
    "improvement_surcharge,"
    "congestion_surcharge,"
    "airport_fee,"
    "total_amount,"
    "payment_type"
)

# Only January 2023 pickups
jan_where = (
    "tpep_pickup_datetime >= '2023-01-01T00:00:00' AND "
    "tpep_pickup_datetime < '2023-02-01T00:00:00'"
)

# Pull a sample of trips
trip_results = client.get(
    TRIP_DATASET_ID,
    select=trip_select,
    where=jan_where,
    limit=100_000
)

df = pd.DataFrame.from_records(trip_results)

print("Rows loaded:", len(df))
print("Columns:", df.columns.tolist())
df.head()


Rows loaded: 100000
Columns: ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'pulocationid', 'dolocationid', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'passenger_count', 'congestion_surcharge', 'airport_fee']


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,pulocationid,dolocationid,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,passenger_count,congestion_surcharge,airport_fee
0,2023-01-01T00:00:00.000,2023-01-01T00:08:00.000,1.53,42,41,12.98,0.0,0.5,0.0,0.0,1.0,14.48,0,NaN,NaN,NaN
1,2023-01-01T00:00:05.000,2023-01-01T00:26:27.000,1.32,249,186,21.9,1.0,0.5,0.0,0.0,1.0,26.9,2,1.0,2.5,0.0
2,2023-01-01T00:00:06.000,2023-01-01T00:05:44.000,1.7,125,68,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2,1.0,2.5,0.0
3,2023-01-01T00:00:08.000,2023-01-01T00:11:24.000,3.1,42,244,16.3,1.0,0.5,0.0,0.0,1.0,18.8,2,1.0,0.0,0.0
4,2023-01-01T00:00:09.000,2023-01-01T00:15:10.000,3.8,79,231,19.8,1.0,0.5,7.44,0.0,1.0,32.24,1,1.0,2.5,0.0


In [167]:
# Convert types and inspect

# Datetime columns
for col in ["tpep_pickup_datetime", "tpep_dropoff_datetime"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Numeric columns
numeric_cols = [
    "passenger_count",
    "trip_distance",
    "pulocationid",
    "dolocationid",
    "fare_amount",
    "extra",
    "mta_tax",
    "tip_amount",
    "tolls_amount",
    "improvement_surcharge",
    "congestion_surcharge",
    "airport_fee",
    "total_amount",
    "payment_type",
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 1   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 2   trip_distance          100000 non-null  float64       
 3   pulocationid           100000 non-null  int64         
 4   dolocationid           100000 non-null  int64         
 5   fare_amount            100000 non-null  float64       
 6   extra                  100000 non-null  float64       
 7   mta_tax                100000 non-null  float64       
 8   tip_amount             100000 non-null  float64       
 9   tolls_amount           100000 non-null  float64       
 10  improvement_surcharge  100000 non-null  float64       
 11  total_amount           100000 non-null  float64       
 12  payment_type           100000 non-null  int64

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,pulocationid,dolocationid,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,passenger_count,congestion_surcharge,airport_fee
count,100000,100000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,96018.000000,96018.000000,96018.000000
mean,2023-01-01 16:45:19.264549888,2023-01-01 17:02:04.725139712,5.024789,159.66582,157.108980,21.859959,1.146657,0.481973,3.567034,0.774770,0.977952,30.557985,1.243190,1.540045,2.172093,0.149738
min,2023-01-01 00:00:00,2023-01-01 00:03:28,0.000000,1.00000,1.000000,-346.000000,-6.000000,-0.500000,-0.900000,-20.750000,-1.000000,-351.000000,0.000000,0.000000,-2.500000,-1.250000
25%,2023-01-01 07:22:40,2023-01-01 07:39:50.750000128,1.200000,114.00000,100.000000,9.300000,0.000000,0.500000,0.000000,0.000000,1.000000,15.120000,1.000000,1.000000,2.500000,0.000000
50%,2023-01-01 15:47:44.500000,2023-01-01 16:06:09,2.210000,158.00000,161.000000,14.200000,1.000000,0.500000,2.520000,0.000000,1.000000,21.000000,1.000000,1.000000,2.500000,0.000000
75%,2023-01-01 23:07:27.249999872,2023-01-01 23:26:10,4.990000,231.00000,233.000000,26.100000,2.500000,0.500000,4.620000,0.000000,1.000000,34.800000,1.000000,2.000000,2.500000,0.000000
max,2023-01-02 13:35:06,2023-01-03 14:01:58,62359.520000,265.00000,265.000000,496.500000,9.750000,1.600000,211.500000,49.850000,1.000000,505.800000,4.000000,6.000000,2.500000,1.250000
std,NaN,NaN,197.458683,66.29284,73.862257,21.330040,1.584788,0.115909,4.535182,2.521912,0.201213,26.173224,0.598291,1.019297,0.899867,0.411472


### Step 2

**Filter to credit card and “typical” trips**

Before looking at tipping patterns, I clean the January 2023 data to focus on trips where the recorded tip information is meaningful and the trip itself is reasonable.

In this step I:

- Keep only trips where `payment_type` indicates credit card (tips are only recorded for card payments).
- Drop trips with clearly invalid values:
  - `fare_amount` ≤ 0  
  - `trip_distance` ≤ 0
- Restrict to a “typical” distance range (for example, between 1 and 20 miles) to reduce the influence of extreme outliers.
- Remove rows with missing values in key columns such as `fare_amount`, `tip_amount`, `total_amount`, and `trip_distance`.

The result is a cleaned subset of January 2023 yellow taxi trips that are paid by credit card and have plausible fares and distances. I will use this filtered DataFrame as the starting point for constructing tip percentages and (later) focusing on Manhattan pickups and time-of-day patterns.


In [168]:
#Filter to credit card + typical trips

print("Rows before filtering:", len(df))

#Keep only credit card trips (TLC code: 1 = credit card)
mask_cc = df["payment_type"] == 1

#Basic validity filters
mask_valid = (
    (df["fare_amount"] > 0) &
    (df["trip_distance"] > 0)
)

#Define a "typical" distance range
mask_typical = (
    (df["trip_distance"] >= 1) &
    (df["trip_distance"] <= 20)
)

#Combine all filters
mask = mask_cc & mask_valid & mask_typical

df_filtered = df[mask].copy()

#Drop rows with missing values in key columns
key_cols = ["fare_amount", "tip_amount", "total_amount", "trip_distance"]
df_filtered = df_filtered.dropna(subset=key_cols)

print("Rows after filtering:", len(df_filtered))
df_filtered.head()


Rows before filtering: 100000
Rows after filtering: 58152


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,pulocationid,dolocationid,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,passenger_count,congestion_surcharge,airport_fee
4,2023-01-01 00:00:09,2023-01-01 00:15:10,3.80,79,231,19.8,1.0,0.5,7.44,0.0,1.0,32.24,1,1.0,2.5,0.00
5,2023-01-01 00:00:13,2023-01-01 00:12:52,8.97,132,7,34.5,1.0,0.5,25.00,0.0,1.0,63.25,1,1.0,0.0,1.25
6,2023-01-01 00:00:18,2023-01-01 00:09:34,2.10,238,262,11.4,3.5,0.5,4.90,0.0,1.0,21.30,1,1.0,2.5,0.00
11,2023-01-01 00:00:57,2023-01-01 00:15:11,9.98,138,97,38.0,6.0,0.5,5.00,0.0,1.0,51.75,1,1.0,0.0,1.25
12,2023-01-01 00:01:07,2023-01-01 00:22:01,3.87,141,68,21.9,1.0,0.5,5.38,0.0,1.0,32.28,1,1.0,2.5,0.00


### Step 3

**Add borough information using the Taxi Zones lookup**

The trip data stores pickup locations as numeric IDs (`pulocationid`), which are not directly interpretable. To identify trips that start in **Manhattan**, I use the NYC Taxi Zones dataset, which maps each `locationid` to a `borough` and `zone`.

In this step I:

- Load the Taxi Zones dataset (`8meu-9t5y`) from the NYC Open Data API.
- Keep only the relevant columns: `locationid`, `borough`, and `zone`.
- Convert `locationid` to a numeric type so it matches `pulocationid` in the trip data.
- Merge the filtered trips from Step 2 with the zones table:
  - Match `pulocationid` from the trips data to `locationid` in the zones data.
  - Create a `pickup_borough` column from the merged `borou_


In [169]:
# Add borough information
ZONE_DATASET_ID = "8meu-9t5y"   # NYC Taxi Zones dataset ID

# 1. Load Taxi Zones lookup from the API
zone_results = client.get(
    ZONE_DATASET_ID,
    limit=500  # more than enough; there are only ~260 zones
)

zones = pd.DataFrame.from_records(zone_results)

print("Taxi Zones columns:", zones.columns.tolist())

#Keep only the relevant columns
zones = zones[["locationid", "borough", "zone"]]

#Ensure types line up for the merge
zones["locationid"] = pd.to_numeric(zones["locationid"], errors="coerce")
df_filtered["pulocationid"] = pd.to_numeric(df_filtered["pulocationid"], errors="coerce")

print("Zones shape:", zones.shape)
zones.head()


Taxi Zones columns: ['the_geom', 'shape_leng', 'shape_area', 'zone', 'locationid', 'borough']
Zones shape: (263, 3)


,locationid,borough,zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


In [170]:
#Merge filtered trips with zones to get pickup borough

df_merged = df_filtered.merge(
    zones,
    left_on="pulocationid",  # from trips
    right_on="locationid",   # from zones
    how="left"
)

#Rename columns for clarity
df_merged = df_merged.rename(
    columns={
        "borough": "pickup_borough",
        "zone": "pickup_zone"
    }
)

#Quick checks
print("Rows in merged DataFrame:", len(df_merged))
print("Unique pickup_borough values:", df_merged["pickup_borough"].unique())

missing_borough = df_merged["pickup_borough"].isna().sum()
print("Rows with missing pickup_borough:", missing_borough)

df_merged.head()


Rows in merged DataFrame: 58160
Unique pickup_borough values: ['Manhattan' 'Queens' nan 'Brooklyn' 'Bronx' 'Staten Island' 'EWR']
Rows with missing pickup_borough: 990


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,pulocationid,dolocationid,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,passenger_count,congestion_surcharge,airport_fee,locationid,pickup_borough,pickup_zone
0,2023-01-01 00:00:09,2023-01-01 00:15:10,3.80,79,231,19.8,1.0,0.5,7.44,0.0,1.0,32.24,1,1.0,2.5,0.00,79.0,Manhattan,East Village
1,2023-01-01 00:00:13,2023-01-01 00:12:52,8.97,132,7,34.5,1.0,0.5,25.00,0.0,1.0,63.25,1,1.0,0.0,1.25,132.0,Queens,JFK Airport
2,2023-01-01 00:00:18,2023-01-01 00:09:34,2.10,238,262,11.4,3.5,0.5,4.90,0.0,1.0,21.30,1,1.0,2.5,0.00,238.0,Manhattan,Upper West Side North
3,2023-01-01 00:00:57,2023-01-01 00:15:11,9.98,138,97,38.0,6.0,0.5,5.00,0.0,1.0,51.75,1,1.0,0.0,1.25,138.0,Queens,LaGuardia Airport
4,2023-01-01 00:01:07,2023-01-01 00:22:01,3.87,141,68,21.9,1.0,0.5,5.38,0.0,1.0,32.28,1,1.0,2.5,0.00,141.0,Manhattan,Lenox Hill West


### Step 4

**Create a tip percentage variable and focus on Manhattan pickups**

To compare tipping behavior across trips, I need a standardized measure of tipping that does not depend on the size of the fare. I therefore convert tips into a percentage of the fare and then narrow the dataset to trips that start in Manhattan, which is where most yellow taxi pickups occur.

In this step I:

- Drop trips with missing or zero `fare_amount` to avoid division by zero.
- Create a new `tip_pct` column defined as `tip_amount / fare_amount`.
- Optionally remove trips with implausible tip percentages (for example, `tip_pct < 0` or `tip_pct > 1.5`) to reduce the influence of outliers.
- Filter the merged data to keep only rows where `pickup_borough` is `"Manhattan"`.

The result is a cleaned DataFrame of Manhattan yellow taxi trips (January 2023, credit card, typical distances) with a `tip_pct` variable that I will use to study how tipping varies by time of day and day of week.


In [171]:
#Create tip_pct and keep Manhattan pickups

# Start from df_merged created in Step 3

# Make sure fare_amount and tip_amount are numeric
df_merged["fare_amount"] = pd.to_numeric(df_merged["fare_amount"], errors="coerce")
df_merged["tip_amount"] = pd.to_numeric(df_merged["tip_amount"], errors="coerce")

# Drop trips with missing or zero fare_amount
before_rows = len(df_merged)
df_merged = df_merged.dropna(subset=["fare_amount", "tip_amount"])
df_merged = df_merged[df_merged["fare_amount"] > 0]

print("Rows after dropping missing/zero fare:", len(df_merged), "(was", before_rows, ")")

# Create tip_pct as tip divided by fare
df_merged["tip_pct"] = df_merged["tip_amount"] / df_merged["fare_amount"]

# Remove implausible tip percentages (e.g., negative or >150%)
mask_plausible = (df_merged["tip_pct"] >= 0) & (df_merged["tip_pct"] <= 1.5)
df_tips = df_merged[mask_plausible].copy()

print("Rows after filtering implausible tip_pct:", len(df_tips))
print(df_tips["tip_pct"].describe())

# Keep only trips that start in Manhattan
df_tips_manhattan = df_tips[df_tips["pickup_borough"] == "Manhattan"].copy()

print("Rows in Manhattan subset:", len(df_tips_manhattan))
df_tips_manhattan[["fare_amount", "tip_amount", "tip_pct", "pickup_borough"]].head()


Rows after dropping missing/zero fare: 58160 (was 58160 )
Rows after filtering implausible tip_pct: 58136
count    58136.000000
mean         0.230928
std          0.107238
min          0.000000
25%          0.184049
50%          0.247170
75%          0.286022
max          1.500000
Name: tip_pct, dtype: float64
Rows in Manhattan subset: 48319


,fare_amount,tip_amount,tip_pct,pickup_borough
0,19.8,7.44,0.375758,Manhattan
2,11.4,4.90,0.429825,Manhattan
4,21.9,5.38,0.245662,Manhattan
5,14.2,0.01,0.000704,Manhattan
6,21.2,6.55,0.308962,Manhattan


### Step 5

**Create time variables for the hourly analysis**

To study how tipping varies over the day for Manhattan trips, I need to extract time information from the pickup timestamp and flag weekday versus weekend rides.

In this step I:

- Use `tpep_pickup_datetime` to create:
  - `pickup_hour` (0–23)
  - `pickup_dayofweek` (0 = Monday, …, 6 = Sunday)
- Define a boolean `is_weekend` indicator (Saturday and Sunday).

These time variables are the foundation for the main visualization in the next step, where I plot the average tip percentage by pickup hour, with separate lines for weekday and weekend trips. In Step 6, I also restrict the analysis to hours that appear in both weekday and weekend data to make the comparison fair.



In [172]:
# Ensure pickup datetime is a proper datetime
df_tips_manhattan["tpep_pickup_datetime"] = pd.to_datetime(
    df_tips_manhattan["tpep_pickup_datetime"],
    errors="coerce"
)

# Extract hour (0–23)
df_tips_manhattan["pickup_hour"] = df_tips_manhattan["tpep_pickup_datetime"].dt.hour

# Day of week (Monday = 0 ... Sunday = 6)
df_tips_manhattan["pickup_dayofweek"] = df_tips_manhattan["tpep_pickup_datetime"].dt.dayofweek

# Weekend flag
df_tips_manhattan["is_weekend"] = df_tips_manhattan["pickup_dayofweek"].isin([5, 6])


### Step 6

**Visualize tipping patterns by hour of day**

To get a clearer view of smaller differences in average tipping, I switch to an hourly view.

In this step I:

- First restrict the data to pickup hours that appear in both weekday and weekend trips, so that the comparison is based on a shared set of hours.
- Compute the average `tip_pct` for each pickup hour in this shared set, separately for weekday and weekend trips.
- Create a line chart of average `tip_pct` vs. `pickup_hour`, with two lines:
  - one for weekday rides
  - one for weekend rides
- Use this chart to see whether there are specific times of day (for example, late evening or night) when weekend tipping is consistently higher than weekday tipping, or whether the two lines largely overlap.

This hourly line plot makes it easier to spot subtle patterns in tipping behavior over the course of the day for Manhattan yellow taxi trips in January 2023, while ensuring that weekday and weekend averages are compared over the same pickup hours.



In [173]:
import plotly.express as px

# Start from the Manhattan subset with tip_pct and time variables
tmp = df_tips_manhattan.dropna(subset=["tip_pct", "pickup_hour", "is_weekend"]).copy()

# Find hours that actually appear for weekdays and weekends
weekday_hours = set(tmp.loc[~tmp["is_weekend"], "pickup_hour"].unique())
weekend_hours = set(tmp.loc[tmp["is_weekend"], "pickup_hour"].unique())

# Intersection = hours that exist in BOTH groups
common_hours = sorted(weekday_hours & weekend_hours)
print("Common hours (used in plot):", common_hours)

# Keep only those common hours
tmp = tmp[tmp["pickup_hour"].isin(common_hours)]

# Group by weekend flag + hour to get average tip_pct
hourly = (
    tmp
    .groupby(["is_weekend", "pickup_hour"], as_index=False)["tip_pct"]
    .mean()
)

# Friendly label for plotting
hourly["day_type"] = hourly["is_weekend"].map({False: "Weekday", True: "Weekend"})

# Line chart: average tip_pct by pickup_hour, weekday vs weekend
fig_line = px.line(
    hourly,
    x="pickup_hour",
    y="tip_pct",
    color="day_type",
    markers=True,
    labels={
        "pickup_hour": "Pickup hour (0–13)",
        "tip_pct": "Average tip percentage (tip_amount / fare_amount)",
        "day_type": "",
    },
    title="Average tip percentage by hour of day (Manhattan pickups, Jan 2023)",
)

fig_line.show()


Common hours (used in plot): [np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12), np.int32(13)]


### Conclusion


The final hourly line chart shows that:

- **Weekend tipping is consistently higher than weekday tipping**, even though  
- **Both weekday and weekend lines follow a very similar pattern across hours 0–13**, rising and falling in almost the same shape.

In other words, riders tend to tip more on weekends than on weekdays, but the way tips change over the course of the day looks broadly similar for both groups. This supports the hypothesis that weekend trips are associated with higher tipping, while suggesting that time-of-day patterns in tipping are structurally similar across weekdays and weekends.
